Here's a lesson on creating Multi-Agent Teams with AutoGen:

# Creating Multi-Agent Teams with AutoGen

## Setup

In [2]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [3]:
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("Async environment configured for Jupyter.")
except ImportError:
    print("Please install nest_asyncio with `pip install nest_asyncio`")

Async environment configured for Jupyter.


## Introduction

Multi-agent teams in AutoGen are groups of AI agents that collaborate to accomplish complex tasks. Teams provide structure for agent interactions and help coordinate their efforts effectively.

## Basic Team Creation

Let's create a simple team with two agents - a primary agent and a critic:

In [4]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini"
)

# Create agents
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a creative writer who writes engaging content."
)

critic_agent = AssistantAgent(
    "critic", 
    model_client=model_client,
    system_message="You provide constructive feedback on writing. Respond with 'APPROVE' when satisfied."
)

# Create termination condition
termination = TextMentionTermination("APPROVE")

# Create team
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=termination
)

## Running the Team

There are two main ways to run a team:

### 1. Basic Run

In [4]:
# Run team synchronously
result = await team.run(task="Write a short story about a magical forest.")
result

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short story about a magical forest.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=31, completion_tokens=850), content='Once upon a time, in a land not so far from here, there existed a magical forest called Eldergrove. It sprawled over rolling hills, wrapping its gnarled roots around ancient trees, each one more magnificent than the last. Sunlight streamed through the dense canopy, casting a dappled glow on the emerald undergrowth below. The air was imbued with the sweet scent of wildflowers and the hushed whispers of secrets held tightly within the trees.\n\nIn this enchanted realm lived Willow, a curious young girl with a wild imagination and a heart full of dreams. With hair the color of autumn leaves and eyes that sparkled like dewdrops on a summer morning, she was drawn to Eldergrove time and time again. Her village lay on the edge of the forest, but t

In [5]:
result.messages

[TextMessage(source='user', models_usage=None, content='Write a short story about a magical forest.', type='TextMessage'),
 TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=31, completion_tokens=850), content='Once upon a time, in a land not so far from here, there existed a magical forest called Eldergrove. It sprawled over rolling hills, wrapping its gnarled roots around ancient trees, each one more magnificent than the last. Sunlight streamed through the dense canopy, casting a dappled glow on the emerald undergrowth below. The air was imbued with the sweet scent of wildflowers and the hushed whispers of secrets held tightly within the trees.\n\nIn this enchanted realm lived Willow, a curious young girl with a wild imagination and a heart full of dreams. With hair the color of autumn leaves and eyes that sparkled like dewdrops on a summer morning, she was drawn to Eldergrove time and time again. Her village lay on the edge of the forest, but the townsfolk warned

### 2. Streaming Run

In [8]:
from autogen_agentchat.base import TaskResult

# Stream messages as they're generated
async for message in team.run_stream(task="Write a short story about a magical forest."):
    if isinstance(message, TaskResult):
        print(f"Task completed: {message.stop_reason}")
    else:
        print(f"{message.source}: {message.content}")


user: Write a short story about a magical forest.
primary: Once, in a realm hidden beyond the ordinary world, there lay a forest known as Lumina Hollow. It was a place where daylight shimmered with a spectrum of colors, and the air was thick with enchantments. The trees, ancient and wise, stood tall with bark that glimmered as if dusted with stardust. It was said that the forest could reveal one's deepest desires if one listened closely to its whispers.

In a nearby village lived a girl named Elara, known for her unruly curls and a heart as adventurous as her spirit. Despite the villagers’ warnings of the forest’s mythical dangers and alluring magic, Elara’s curiosity was an unquenchable flame. One evening, as the sky blushed with hues of orange and pink, she made her decision. With a deep breath and a heart full of anticipation, she slipped away from the village and ventured into Lumina Hollow.

The moment Elara crossed the threshold into the forest, she felt a rush of warmth envelop 

## Team Types

AutoGen provides several team types for different use cases:

### 1. RoundRobinGroupChat
Agents take turns in a fixed order. Good for structured discussions.

In [5]:
agent1 = AssistantAgent(
    "Writer",
    model_client=model_client,
    system_message="You are a helpful assistant."
)

agent2 = AssistantAgent(
    "Editor",
    model_client=model_client,
    system_message="You provide constructive feedback on writing. Respond with 'APPROVE' when satisfied."
)

team = RoundRobinGroupChat(
    [agent1, agent2],
    termination_condition=termination
)

output = await team.run(task="Write 3 jokes about the tragedy of life.")
output

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write 3 jokes about the tragedy of life.', type='TextMessage'), TextMessage(source='Writer', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=123), content='Sure! Here are three jokes that touch on the tragedy of life with a humorous twist:\n\n1. Why did the nihilist refuse to play cards?\n   Because he couldn\'t deal with the hand life dealt him!\n\n2. Life is like a rollercoaster—full of ups and downs. \n   But nobody warned me it would leave me feeling queasy and screaming the whole way!\n\n3. They say life is a journey, but mine feels more like a GPS with no signal— \n   always making wrong turns and asking if I want to reroute to "disappointment"!\n\nHope you enjoyed these!', type='TextMessage'), TextMessage(source='Editor', models_usage=RequestUsage(prompt_tokens=167, completion_tokens=4), content='APPROVE', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

# SelectorGroupChat 
[`SelectorGroupChat`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.SelectorGroupChat) is a group chat that uses an AI model to intelligently choose the next speaker based on context.

 How it works:
 - When given a task via run() or run_stream():
   1. The model analyzes conversation history and agent profiles to pick the next speaker
   2. Selected agent provides a response that's shared with all participants 
   3. Process repeats until termination condition is met

 Key features:
 - By default, prevents same speaker twice in a row (override with allow_repeated_speaker=True)
 - Can use custom selection function instead of default model
 - Maintains conversation context between tasks
 - Call reset() to clear conversation history
 - Returns TaskResult with full conversation when complete

### 2. SelectorGroupChat 
Uses an AI model to dynamically choose the next speaker based on context.

In [7]:
from autogen_agentchat.teams import SelectorGroupChat

agent1 = AssistantAgent(
    "Writer",
    model_client=model_client,
    system_message="You are a creative joke writer."
)

agent2 = AssistantAgent(
    "Editor",
    model_client=model_client,
    system_message="You take in the score from the Ranker and provide feedback on the jokes in case they are below 5, if not you return with 'APPROVE'."
)

agent3 = AssistantAgent(
    "Ranker",
    model_client=model_client,
    system_message="You rank the quality of the jokes, giving an overall score from 1 to 10."
)

team = SelectorGroupChat(
    participants=[agent1, agent2, agent3],
    model_client=model_client,
    termination_condition=termination
)

In [8]:
output = await team.run(task="Write 3 jokes about the tragedy of life.")

In [10]:
output.messages

[TextMessage(source='user', models_usage=None, content='Write 3 jokes about the tragedy of life.', type='TextMessage'),
 TextMessage(source='Writer', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=127), content='Sure! Here are three jokes that touch on the tragedy of life with a touch of humor:\n\n1. Why did the existentialist refuse to play hide and seek?\n   Because good luck hiding from the reality of life!\n\n2. I told my friend I was feeling down about the tragedy of life\'s fleeting moments.\n   He said, "At least they’re fleeting—some people have a habit of dragging it out longer than a soap opera!"\n\n3. Life\'s like a tragic opera: it has its ups and downs, a lot of drama, and just when you think it’s over—surprise! There’s a sequel!', type='TextMessage'),
 TextMessage(source='Editor', models_usage=RequestUsage(prompt_tokens=187, completion_tokens=179), content="The jokes you provided have a clever twist, but they might not resonate with everyone and could be se

### 3. Swarm

Agents communicate through explicit handoffs, allowing targeted interactions.

See the [docs for more infor on this advanced multi-agent pattern.](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/swarm.html) 

## Best Practices

1. Choose the right team type for your use case:
   - RoundRobinGroupChat for structured, turn-based interactions
   - SelectorGroupChat for dynamic, context-aware speaker selection
   - Swarm for explicit handoffs between agents

2. Use appropriate termination conditions:
   - TextMentionTermination for keyword-based stopping
   - MaxMessageTermination for limiting conversation length
   - ExternalTermination for manual control

3. Reset teams between unrelated tasks to clear conversation history

4. Use streaming for real-time monitoring of team interactions

5. Start with simple teams and add complexity as needed

Remember that teams add overhead compared to single agents, so use them only when the task complexity justifies multi-agent collaboration.